In [66]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
import joblib

In [67]:
# 1. Chargement des données
data = pd.read_csv("D:\\M2 SISE\\Python Machine Learning\\Projet\\Performance_Energetique\\Data\\data69rhone.csv")  # Remplacez par le chemin de votre fichier si nécessaire

C:\Users\karam\AppData\Local\Temp\ipykernel_11300\1771313095.py:2: DtypeWarning: Columns (3,27,39,106) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("D:\\M2 SISE\\Python Machine Learning\\Projet\\Performance_Energetique\\Data\\data69rhone.csv")  # Remplacez par le chemin de votre fichier si nécessaire


In [68]:
# Imputation pour les colonnes avec peu de valeurs manquantes
for col in data.select_dtypes(include=['float64', 'int64']).columns:
    data[col].fillna(data[col].median(), inplace=True)
    
for col in data.select_dtypes(include=['object']).columns:
    data[col].fillna(data[col].mode()[0], inplace=True)

C:\Users\karam\AppData\Local\Temp\ipykernel_11300\1086663217.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].median(), inplace=True)
C:\Users\karam\AppData\Local\Temp\ipykernel_11300\1086663217.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

In [69]:

data['Besoin_ECS'] = pd.to_numeric(data['Besoin_ECS'].replace(',', '.', regex=True), errors='coerce')

In [70]:
# Supposons que la colonne cible est "Etiquette_DPE" et qu'il y a des colonnes catégorielles et numériques
target = 'Etiquette_DPE'
data = data.dropna(subset=[target])

In [71]:
ls_variables_explicatives = [
'Conso_5_usages_par_m²_é_primaire',
'Emission_GES_5_usages_par_m²',
'Conso_5_usages/m²_é_finale',
'Conso_chauffage_é_primaire',
'Emission_GES_chauffage',
'Coût_chauffage',
'Besoin_ECS',
'Surface_habitable_logement',
'Coût_total_5_usages',
'Conso_5_usages_é_finale_énergie_n°1',
'Conso_ECS_é_primaire',
'Conso_éclairage_é_finale',
'Type_énergie_principale_chauffage',
'Conso_chauffage_é_finale',
'Année_construction'
]

In [72]:
for  i  in  ls_variables_explicatives:
    print(i)
    print(data[i].unique())

Conso_5_usages_par_m²_é_primaire
[535.  583.  386.  ...  20.2   7.7  26.3]
Emission_GES_5_usages_par_m²
[1.700e+01 1.900e+01 8.400e+01 ... 1.603e+02 7.000e-01 8.562e+02]
Conso_5_usages/m²_é_finale
[232.9 253.  378.  ...  12.6  11.5  11.8]
Conso_chauffage_é_primaire
[11063.2 19475.9 13198.5 ...  1830.8  3596.1  4790.8]
Emission_GES_chauffage
[  380.    669.   2996.1 ... 10756.3 12863.3 15140.1]
Coût_chauffage
[1040.  1766.1 1413.4 ... 7096.5 8370.8 4505. ]
Besoin_ECS
[ 848.1 1022.5 1006.  ...  734.   621.   552. ]
Surface_habitable_logement
[  27.    41.5   40.1 ...  398.9 1506.2  231.5]
Coût_total_5_usages
[ 1360.   2195.5  1684.8 ... 14490.1 10904.   8377. ]
Conso_5_usages_é_finale_énergie_n°1
[ 6287.8 10526.9 14907.3 ...  2511.6  1801.3  1939.2]
Conso_ECS_é_primaire
[ 3283.3  3801.7  1708.7 ...   875.2 22524.4  1032.7]
Conso_éclairage_é_finale
[   50.2    77.1    74.6 ...  3159.4 19053.2  1823.8]
Type_énergie_principale_chauffage
['Électricité' 'Gaz naturel' 'Charbon' 'Bois – Bûches'

In [73]:
# Définir les données d'entrée et la variable cible
X = data[ls_variables_explicatives]
y = data[target]
# Séparer les colonnes numériques et catégorielles
numeric_features = data[ls_variables_explicatives].select_dtypes(include=[np.number]).columns
categorical_features = data[ls_variables_explicatives].select_dtypes(exclude=[np.number]).columns


In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [75]:
data[target]

0         G
1         G
2         F
3         D
4         C
         ..
287160    A
287161    A
287162    A
287163    A
287164    A
Name: Etiquette_DPE, Length: 287165, dtype: object

In [ ]:
# Création des transformations pour chaque type de données
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),      # Impute les valeurs manquantes par la moyenne
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute les valeurs manquantes par la valeur la plus fréquente
    ('ordinal', OrdinalEncoder())                          # Utilise OrdinalEncoder pour encoder les variables catégorielles
])

# Application des transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# 4. Création du pipeline complet
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),  
    ('classifier', RandomForestClassifier(random_state=42))
])

In [77]:
# 6. Entraînement du modèle
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  Index(['Conso_5_usages_par_m²_é_primaire', 'Emission_GES_5_usages_par_m²',
       'Conso_5_usages/m²_é_finale', 'Conso_chauffage_é_primaire',
       'Emission_GES_chauffage', 'Coût_chauffage', 'Besoin_ECS',
       'Surface_habitable_logement', 'Coût_total_5_usag...
       'Conso_5_usages_é_finale_énergie_n°1', 'Conso_ECS_é_primaire',
       'Conso_éclairage_é_finale', 'Conso_chauffage_é_finale',
       'Année_construction'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder())]),
                                                  Index(['Type_énergie_principale_chauffage'], dtype='object'))])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [78]:
y_pred = pipeline.predict(X_test)
y_pred

array(['D', 'C', 'D', ..., 'C', 'C', 'D'], dtype=object)

In [79]:
# 7. Évaluation du modèle
print("Matrice de confusion :\n", confusion_matrix(y_test, y_pred))
print("Rapport de classification :\n", classification_report(y_test, y_pred))

Matrice de confusion :
 [[ 1525    11     0     1     0     0     0]
 [    1  3303    30     0     0     0     0]
 [    0     0 32162   209     2     0     0]
 [    0     1   201 25487   169     3     1]
 [    0     0    12   178 14673    90     2]
 [    0     0     2    14    83  4832    46]
 [    0     0     5     6    14    43  3044]]
Rapport de classification :
               precision    recall  f1-score   support

           A       1.00      0.99      1.00      1537
           B       1.00      0.99      0.99      3334
           C       0.99      0.99      0.99     32373
           D       0.98      0.99      0.98     25862
           E       0.98      0.98      0.98     14955
           F       0.97      0.97      0.97      4977
           G       0.98      0.98      0.98      3112

    accuracy                           0.99     86150
   macro avg       0.99      0.98      0.99     86150
weighted avg       0.99      0.99      0.99     86150



In [80]:
# 9. Chargement et utilisation du pipeline sauvegardé (optionnel)
pipeline_loaded = joblib.load('pipeline.pkl')
y_pred_loaded = pipeline_loaded.predict(X_test)

In [81]:
#Step 10: Load the pipeline from the .pkl file
pipeline_loaded = joblib.load('pipeline.pkl')  # Load the pipeline from the .pkl file
print("Pipeline loaded from pipeline.pkl")

Pipeline loaded from pipeline.pkl


In [82]:
#Step 11: Use the loaded pipeline to make predictions
y_pred_loaded = pipeline_loaded.predict(X_test)
print("Évaluation du pipeline chargé :")
print("Matrice de confusion :\n", confusion_matrix(y_test, y_pred_loaded))
print("Rapport de classification :\n", classification_report(y_test, y_pred_loaded))

Évaluation du pipeline chargé :
Matrice de confusion :
 [[ 1529     7     0     1     0     0     0]
 [    1  3314    19     0     0     0     0]
 [    0     1 32239   131     2     0     0]
 [    0     2   151 25588   120     0     1]
 [    0     0     9   110 14782    54     0]
 [    0     0     2    11    55  4881    28]
 [    0     0     1     5    12    22  3072]]
Rapport de classification :
               precision    recall  f1-score   support

           A       1.00      0.99      1.00      1537
           B       1.00      0.99      1.00      3334
           C       0.99      1.00      1.00     32373
           D       0.99      0.99      0.99     25862
           E       0.99      0.99      0.99     14955
           F       0.98      0.98      0.98      4977
           G       0.99      0.99      0.99      3112

    accuracy                           0.99     86150
   macro avg       0.99      0.99      0.99     86150
weighted avg       0.99      0.99      0.99     86150

